# HackerNews data analysis challenge with Spark

In this notebook, you will analyse a dataset of (almost) all submitted HackerNews posts with Spark. Let's start by importing some of the libraries you will need.

In [1]:
import json
import numpy as np
import matplotlib.pyplot as plt
from datetime import datetime as dt
%matplotlib inline

In [2]:
from pyspark import SparkContext
sc = SparkContext.getOrCreate()
print(sc)
print("Ready to go!")

<SparkContext master=local[*] appName=pyspark-shell>
Ready to go!


The file has one JSON entry per line. In order to make accessing it easier, first turn each entry as a dictionary and use `persist()` to cache the resulting RDD.

In [3]:
dataset_json = sc.textFile("HNStories.json")
dataset = dataset_json.map(lambda x: json.loads(x))
dataset.persist()

PythonRDD[2] at RDD at PythonRDD.scala:48

Finally, Spark has many helper functions on top of the ones we have studied which you will find useful. You can view them at [http://spark.apache.org/docs/latest/api/python/pyspark.html#pyspark.RDD](http://spark.apache.org/docs/latest/api/python/pyspark.html#pyspark.RDD)

### Task 1

Lets start with some initial analysis. 
* How many elements are in your datasets?
* What does the first element look like?

In [4]:
dataset.take(1)

[{'author': 'TuxLyn',
  'created_at': '2014-05-29T08:25:40Z',
  'created_at_i': 1401351940,
  'num_comments': 0,
  'objectID': '7815290',
  'points': 1,
  'title': 'DuckDuckGo Settings',
  'url': 'https://duckduckgo.com/settings'}]

In [5]:
print('Number of elements in each data', len(dataset.take(1)[0]))

Number of elements in each data 8


Each element is a dictionary of attributes and their values for a post. Can you find the set of all attributes used throughout the RDD? The function `dictionary.keys()` gives you the list of attributes of a dictionary.

In [6]:
dataset.take(1)[0].keys()

dict_keys(['author', 'created_at', 'created_at_i', 'num_comments', 'objectID', 'points', 'title', 'url'])

In [8]:
elements = dataset.flatMap(lambda x: x.keys()).distinct().collect()
print(elements)

['created_at', 'points', 'created_at_i', 'story_text', 'objectID', 'title', 'author', 'url', 'story_id', 'num_comments']


We see that there are more attributes than just the one used in the first element. the function `compare_elems` below returns `True` if two elements have exactly the same set of attributes. Can you use it to count the number of elements which have the same set of attributes as the first element?

In [13]:
# Returns true if two elements have the same schema
#first, second is a list

def compare_elems(first, second):
    if len(first) != len(second):
        return False
    for key in first:
        if key not in second:
            return False    
    return True

In [18]:
first = dataset.take(1)
compared_data = dataset.map(lambda x: compare_elems(first[0].keys(), x.keys()))
print('Number of elements which have the same set of attributes as the first element is: ',
      compared_data.filter(lambda x: x==True).count())

Number of elements which have the same set of attributes as the first element is:  1145245


We see that the vast majority of elements hold the same structure. In order to make this analysis easier, redefine `dataset` to only have elements which hold this structure.

In [20]:
new_dataset = dataset.filter(lambda x: compare_elems(first[0].keys(), x.keys()))
print('The number of elements in new dataset: ', new_dataset.count())
new_dataset.take(3)

The number of elements in new dataset:  1145245


[{'author': 'TuxLyn',
  'created_at': '2014-05-29T08:25:40Z',
  'created_at_i': 1401351940,
  'num_comments': 0,
  'objectID': '7815290',
  'points': 1,
  'title': 'DuckDuckGo Settings',
  'url': 'https://duckduckgo.com/settings'},
 {'author': 'Leynos',
  'created_at': '2014-05-29T08:23:46Z',
  'created_at_i': 1401351826,
  'num_comments': 0,
  'objectID': '7815285',
  'points': 1,
  'title': 'Making Twitter Easier to Use',
  'url': 'http://bits.blogs.nytimes.com/2014/05/28/making-twitter-easier-to-use/'},
 {'author': 'darrhiggs',
  'created_at': '2014-05-29T08:21:56Z',
  'created_at_i': 1401351716,
  'num_comments': 0,
  'objectID': '7815279',
  'points': 2,
  'title': 'London refers Uber app row to High Court',
  'url': 'http://www.bbc.co.uk/news/technology-27617079'}]

All of the following tasks are optional, if you want to analyse the dataset in your own way using Spark feel free to do so! The tasks are there as a guide.

### Task 2: How many posts through time

The field `created_at_i` is very useful, it gives you a UNIX timestamp of the time at which the file was created. The following function lets you extract a time from a timestamp.

In [21]:
def extract_time(timestamp):
    return dt.fromtimestamp(timestamp)

Find the minimum and maximum timestamps in the RDD and call them `min_time` and `max_time`. These correspond to the first and last post, when did they occur?

In [29]:
def max(x, y):
    if x['created_at_i'] > y['created_at_i']:
        return x
    else:
        return y

In [30]:
mnew_dataset.reduce(lambda x, y: max(x, y))['created_at_i']

{'author': 'TuxLyn',
 'created_at': '2014-05-29T08:25:40Z',
 'created_at_i': 1401351940,
 'num_comments': 0,
 'objectID': '7815290',
 'points': 1,
 'title': 'DuckDuckGo Settings',
 'url': 'https://duckduckgo.com/settings'}

In [ ]:
print(min_time)
print(max_time)

Now lets analyse how many elements through time. The following function assigns a record to one of 200 "buckets" of time. Use it to count the number of elements that fall within each bucket and call the result `bucket_rdd`. The result should be such that `buckets` below generates the corresponding output.

In [ ]:
interval = (max_time - min_time + 1) / 200.0

def get_bucket(rec):
    return int((rec['created_at_i'] - min_time) / interval)

In [ ]:
# Use this to test your result
buckets = sorted(buckets_rdd.collect())
print(buckets)

In [ ]:
# This is approximately the desired output
buckets = sorted(buckets_rdd.collect())
print(buckets)

We can then use this to plot the number of submitted posts over time.

In [ ]:
bs = [dt.fromtimestamp(x[0]*interval + min_time) for x in buckets]
ts = [x[1] for x in buckets]
plt.plot(bs, ts)

### Task 3

The following function gets the hour of the day at which a post was submitted. Use it to find the number of posts submitted at each hour of the day. The value of `hours_buckets` should match the one printed below.

In [ ]:
def get_hour(rec):
    t = dt.fromtimestamp(rec['created_at_i'])
    return t.hour

In [ ]:
hours_buckets = sorted(hours_buckets_rdd.collect())
print(hours_buckets)

In [ ]:
hours_buckets = sorted(hours_buckets_rdd.collect())
print(hours_buckets)

In [ ]:
hrs = [x[0] for x in hours_buckets]
sz = [x[1] for x in hours_buckets]
plt.plot(hrs, sz)

### Task 4

The number of points scored by a post is under the attribute `points`. Use it to compute the average score received by submissions for each hour.

In [ ]:
#what the data looks like
dataset.take(1)

In [ ]:
dt.

In [ ]:
scores_per_hour = sorted(scores_per_hour_rdd.collect())
print(scores_per_hour)

In [ ]:
scores_per_hour = sorted(scores_per_hour_rdd.collect())
print(scores_per_hour)

In [ ]:
hrs = [x[0] for x in scores_per_hour]
sz = [x[1] for x in scores_per_hour]
plt.plot(hrs, sz)

It may be more useful to look at sucessful posts that get over 200 points. Find the proportion of posts that get above 200 points per hour.

In [ ]:
prop_per_hour = sorted(prop_per_hour_rdd.collect())
print(prop_per_hour)

In [ ]:
prop_per_hour = sorted(prop_per_hour_rdd.collect())
print(prop_per_hour)

In [ ]:
hrs = [x[0] for x in prop_per_hour]
sz = [x[1] for x in prop_per_hour]
plt.plot(hrs, sz)

### Task 5

The following function lists the word in the title. Use it to count the number of words in the title of each post, and look at the proportion of successful posts for each title length.

In [ ]:
import re
def get_words(line):
    return re.compile('\w+').findall(line)

In [ ]:
prop_per_title_length = sorted(prop_per_title_length_rdd.collect())
print(prop_per_title_length)

In [ ]:
prop_per_title_length = sorted(prop_per_title_length_rdd.collect())
print(prop_per_title_length)

In [ ]:
hrs = [x[0] for x in prop_per_title_length]
sz = [x[1] for x in prop_per_title_length]
plt.plot(hrs, sz)

Lets compare this with the distribution of number of words. Count for each title length the number of submissions with that length.

In [ ]:
submissions_per_length = sorted(submissions_per_length_rdd.collect())
print(submissions_per_length)

In [ ]:
submissions_per_length = sorted(submissions_per_length_rdd.collect())
print(submissions_per_length)

In [ ]:
hrs = [x[0] for x in submissions_per_length]
sz = [x[1] for x in submissions_per_length]
plt.plot(hrs, sz)

Looks like most people are getting it wrong!

### Task 6

For this task, you will need a new function: `takeOrdered()`. Like `take()` it collects elements from an RDD. However, it can be applied the smalles elements. For example, `takeOrdered(10)` returns the 10 smallest elements. Furthermore, you can pass it a function to specify the way in which the elements should be ordered. For example, `takeOrdered(10, lambda x: -x)` will return the 10 largest elements.

The function below extracts the url domain out of a record. Use it to count the number of distinct domains posted to.

In [ ]:
from urlparse import urlparse
def get_domain(rec):
    url = urlparse(rec['url']).netloc
    if url[0:4] == 'www.':
        return url[4:]
    else:
        return url
print(get_domain(dataset.take(1)[0]))

Using `takeOrdered()` find the 25 most popular domains posted to.

In [ ]:
print(top25)

In [ ]:
print(top25)

In [ ]:
index = np.arange(25)
labels = [x[0] for x in top25]
counts = np.array([x[1] for x in top25]) * 100.0/dataset.count()
plt.xticks(index,labels, rotation='vertical')
plt.bar(index, counts, 0.5)

Create an pair RDD with 26 elements mapping each of these 25 popular domains with the average score received by the corresponding submissions as well as an `other` field for all submissions to other domains.

In [ ]:
def map_to_domain(rec):
    domain = get_domain(rec)
    if domain in dict(top25):
        return domain
    else:
        return 'other'

In [ ]:
domain_av_score = domain_av_score_rdd.collect()
print(domain_av_score)

In [ ]:
domain_av_score = domain_av_score_rdd.collect()
print(domain_av_score)

In [ ]:
index26 = np.arange(26)
labels = [x[0] for x in top25]
labels.append('other')
vals = np.array([dict(domain_av_score)[x] for x in labels])
plt.xticks(index26, labels, rotation='vertical')
plt.bar(index26, vals, 0.5)

Now compute the proportion of successes for each domain (over 200 points).

In [ ]:
domain_prop = domain_prop_rdd.collect()
print(domain_prop)

In [ ]:
domain_prop = domain_prop_rdd.collect()
print(domain_prop)

In [ ]:
index26 = np.arange(26)
labels = [x[0] for x in top25]
labels.append('other')
vals = np.array([dict(domain_prop)[x] for x in labels])
plt.xticks(index26, labels, rotation='vertical')
plt.bar(index26, vals, 0.5)